In [1]:
import pickle
import pandas as pd
X_hr2,y_stress2,y_participant2,X_time2 = pickle.load(open('../data/tabular_data_60_seconds_ppg_rr_dalia_normalized.p','rb'))

X_hr2 = X_hr2.reshape(-1,30,1)

import pickle
X_hr1,y_stress1,y_participant1,X_time1 = pickle.load(open('../data/tabular_data_60_seconds_ppg_rr_normalized.p','rb'))

X_hr1 = X_hr1.reshape(-1,30,1)

import numpy as np
X_hr = np.concatenate([X_hr2,X_hr1])

y_participant = np.array(list(y_participant2)+list(y_participant1+np.max(y_participant1)+1))

y_time = np.array(list(y_stress2)+list(y_stress1))

In [2]:
X_hr,y_time,y_participant,X_acl = pickle.load(open('../data/tabular_data_60_seconds_ppg_rr_dalia_normalized.p','rb'))
X_hr = X_hr.reshape(-1,30,1)
X_acl = X_acl.reshape(-1,30,1)
# X_hr = np.concatenate([X_hr,X_acl],axis=-1)

In [3]:
all_data = pd.DataFrame({'x':np.arange(X_hr.shape[0]),'y':y_participant,'time':y_time})

train_percentage = .6
val_percentage = train_percentage+.1
def split_data(df):
    df = df.sort_values('time').reset_index(drop=True)
    n = df.shape[0]
    train_index = df['x'].values[:int(n*train_percentage)]
    val_index = df['x'].values[int(n*train_percentage):int(n*val_percentage)]
    test_index = df['x'].values[int(n*val_percentage):]
    return pd.DataFrame({'train':[list(train_index)],'val':[list(val_index)],'test':[list(test_index)]})

index_df = all_data.groupby('y',as_index=False).apply(split_data)

from functools import reduce
train_index = np.array(reduce(lambda a,b:a+b,index_df['train'].values))
val_index = np.array(reduce(lambda a,b:a+b,index_df['val'].values))
test_index = np.array(reduce(lambda a,b:a+b,index_df['test'].values))


train_x,train_y = X_hr[train_index],y_participant[train_index]
val_x,val_y = X_hr[val_index],y_participant[val_index]
test_x,test_y = X_hr[test_index],y_participant[test_index]

In [4]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,LeaveOneGroupOut,LeavePGroupsOut
from sklearn.metrics import accuracy_score
from tensorflow import keras
from tensorflow.keras import backend as K 
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import numpy as np
%matplotlib notebook
from keras.layers import Conv1D,Reshape,BatchNormalization,TimeDistributed, \
Dropout,Input,MaxPooling1D,Flatten,Dense,Input, GaussianNoise,LSTM, Bidirectional, Input
from keras.models import Model, Sequential
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer,LabelEncoder
from sklearn.metrics.pairwise import euclidean_distances
from scipy.stats.mstats import winsorize
def get_model(input_shape=(256,4),act='relu',loss="categorical_crossentropy",opt='adam',
              n_classes=350,n_output = 15):
    
    
    model =  Sequential()
    model.add(Input(shape=input_shape))
    model.add(Reshape(input_shape))
    model.add(Conv1D(100,40,input_shape=input_shape,activation='selu',kernel_initializer='normal',padding='same'))
#     model.add(Bidirectional(GRU(10,activation='relu',return_sequences=True)))
    model.add(Conv1D(50,40,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
#     model.add(Bidirectional(GRU(10,activation='relu',return_sequences=True)))
#     model.add(BatchNormalization())
#     model.add(Dropout(.2))
    model.add(Conv1D(100,20,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
#     model.add(Bidirectional(GRU(10,activation='relu',return_sequences=True)))
    model.add(BatchNormalization())
#     model.add(Dropout(.2))
    model.add(Conv1D(100,20,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
#     model.add(Bidirectional(GRU(10,activation='relu',return_sequences=True)))
    model.add(BatchNormalization())
#     model.add(Conv1D(100,20,activation='relu',kernel_initializer='normal',padding='same'))
#     model.add(MaxPooling1D(2))
# #     model.add(Bidirectional(GRU(10,activation='relu',return_sequences=True)))
#     model.add(BatchNormalization())
    
#     model.add(Conv1D(100,20,activation='relu',kernel_initializer='normal',padding='same'))
#     model.add(MaxPooling1D(2))
#     model.add(BatchNormalization())
    
#     model.add(Conv1D(200,2,activation='relu',kernel_initializer='normal',padding='same'))
#     model.add(BatchNormalization())
#     model.add(Bidirectional(GRU(10,activation='relu',return_sequences=True)))
    model.add(Dropout(.2))
    model.add(Flatten())
#     model.add(Dense(n_classes,activation='relu'))
#     model.add(Dense(n_output*2,activation='relu'))
    model.add(Dense(n_output,activation=None))
    model.summary()
    input_ = Input(shape=input_shape)
    reshaped_input = Reshape(input_shape)(input_)
    embedding = model(reshaped_input)
    embedding = tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1),name='feature')(embedding)
    
#     class_embedding = Dense(n_output,activation='sigmoid')(final_embedding)
    y_output = Dense(n_output,activation='softmax',name='output')(embedding) 
    model1 = Model(input_,y_output)
#     model1.compile(loss={'output':tf.keras.losses.SparseCategoricalCrossentropy(),
#                         'feature':tfa.losses.TripletSemiHardLoss()},optimizer=opt,
#                    loss_weights={'output':1,'feature':0})
    model1.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),optimizer='adam')
    return model1

# model1 = get_model(input_shape=(n_timesteps,1))
# model1.summary()
n_output = len(np.unique(train_y))
model = get_model(input_shape=(30,1),n_classes=n_output,n_output=n_output) 
model.summary()

4 Physical GPUs, 1 Logical GPU
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 30, 1)             0         
_________________________________________________________________
conv1d (Conv1D)              (None, 30, 100)           4100      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 30, 50)            200050    
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 15, 50)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 15, 100)           100100    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 7, 100)            0         
_________________________________________________________________
batch_normalization (Batc

In [5]:
result = []
for k in [0]:
    n_classes = 200
    n_output = len(np.unique(y_participant))
    model = get_model(input_shape=(30,1),n_classes=n_output,n_output=n_output) 
    model.summary()
    from keras.models import load_model
    filepath = '../model_files/base_cnn_60_seconds_ppg_hr_wesad_dalia_combined_only_classification1.hdf5'

    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min',save_weights_only=True)
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=40)
    callbacks_list = [es,checkpoint]
    history = model.fit(train_x,train_y,validation_data=(val_x,val_y), epochs=400, 
                        batch_size=500,callbacks=callbacks_list,shuffle=True,verbose=1)
    

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_2 (Reshape)          (None, 30, 1)             0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 30, 100)           4100      
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 30, 50)            200050    
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 15, 50)            0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 15, 100)           100100    
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 7, 100)            0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 7, 100)           

In [6]:
model.load_weights(filepath)
test_y_pred = model.predict(test_x).argmax(axis=1)

from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(k,accuracy_score(test_y,test_y_pred))
print(classification_report(test_y,test_y_pred))

0 0.15034747432838916
              precision    recall  f1-score   support

           0       0.07      0.08      0.07      1348
           1       0.29      0.17      0.21      1363
           2       0.01      0.00      0.00      1181
           3       0.09      0.02      0.03      1302
           4       0.14      0.36      0.20      1392
           5       0.27      0.21      0.24      1178
           6       0.00      0.00      0.00       778
           7       0.08      0.05      0.06      1361
           8       0.20      0.12      0.15      1588
           9       0.20      0.51      0.29      1386
          10       0.15      0.29      0.20      1275
          11       0.15      0.02      0.03      1203
          12       0.22      0.04      0.07      1221
          13       0.09      0.05      0.06      1334
          14       0.13      0.23      0.17      1372

    accuracy                           0.15     19282
   macro avg       0.14      0.14      0.12     19282
weig

In [7]:
result= np.array(result)

In [ ]:
%matplotlib inline
plt.rcParams.update({'font.size':25})
plt.figure(figsize=(10,8))
plt.plot(result[:,0],result[:,1],'o--r')
plt.xlabel('Winsorization Limit')
plt.ylabel('Reidentification Accuracy')
plt.show()

In [ ]:
%matplotlib inline
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
train_embeddings_pca = TSNE(n_components=2).fit_transform(test_embeddings)
plt.figure()
plt.scatter(train_embeddings_pca[:,0],train_embeddings_pca[:,1],c=test_y)
plt.colorbar()
plt.show()

In [ ]:
import pickle
test_dalia_x,test_dalia_y_stress,test_dalia_y,test_dalia_time = pickle.load(open('../data/tabular_data_60_seconds_ppg_rr_dalia_normalized.p',
                                                                                 'rb'))

In [ ]:
test_dalia_x = test_dalia_x.reshape(-1,30,1)

In [ ]:
test_dalia_embeddings = model.predict(test_dalia_x)

In [ ]:
train_embeddings_pca = TSNE(n_components=2).fit_transform(test_dalia_embeddings)
plt.figure()
plt.scatter(train_embeddings_pca[:,0],train_embeddings_pca[:,1],c=test_dalia_y)
plt.colorbar()
plt.show()

In [ ]:
model = get_model(input_shape=(30,1),n_classes=n_classes,n_output=n_output,loss=custom_loss) 
model.summary()
model.load_weights(filepath)

In [ ]:
y_pred = model.predict(test_x).argmax(axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
print(confusion_matrix(test_y,y_pred))

In [ ]:
%matplotlib inline
def get_clusters(model,train_x,train_y):
    train_embeddings = model.predict(train_x)

    from sklearn.decomposition import PCA

    train_embeddings_pca = PCA(n_components=20).fit_transform(train_embeddings)

    plt.figure()
    plt.scatter(train_embeddings_pca[:,0],train_embeddings_pca[:,1],c=train_y)
    plt.show()
get_clusters(model,test_x,test_y)

In [ ]:
import pickle
test_dalia_x,test_dalia_y_stress,test_dalia_y,test_dalia_time = pickle.load(open('../data/tabular_data_60_seconds_ppg_rr_dalia.p',
                                                                                 'rb'))
test_dalia_x = test_dalia_x.reshape(-1,30,1)
get_clusters(model,test_dalia_x,test_dalia_y)

In [ ]:
def give_rank_accuracy(train_embeddings,train_y,rank = 10):
    distance_matrix = euclidean_distances(train_embeddings)
    distance_matrix_2d = np.zeros((distance_matrix.shape[0],
                               distance_matrix.shape[0],
                               2))
    distance_matrix_2d[:,:,0] = distance_matrix
    train_y_matrix = np.concatenate([train_y.reshape(1,-1)]*distance_matrix.shape[0])
    distance_matrix_2d[:,:,1] = train_y_matrix
    match = 0
    for i in range(distance_matrix.shape[0]):
        row = distance_matrix_2d[i]
        row = row[row[:,0].argsort(),:]
        if rank==1:
            if train_y[i]==row[1,1]:
                match+=1
        else:
            row = set(row[1:(rank+1)][:,1])
            if train_y[i] in row:
                match+=1
    return match/distance_matrix.shape[0]

In [ ]:
rank_1 = give_rank_accuracy(train_embeddings,train_y,rank = 1)
rank_5 = give_rank_accuracy(train_embeddings,train_y,rank = 5)
rank_10 = give_rank_accuracy(train_embeddings,train_y,rank = 10)
print(rank_1,rank_5,rank_10)

In [ ]:
test_embeddings = model.predict(test_x)

In [ ]:
rank_1 = give_rank_accuracy(test_embeddings,test_y,rank = 1)
rank_5 = give_rank_accuracy(test_embeddings,test_y,rank = 5)
rank_10 = give_rank_accuracy(test_embeddings,test_y,rank = 10)
print(rank_1,rank_5,rank_10)

In [ ]:
test_x,y_stress,test_y,X_time = pickle.load(open('../data/tabular_data_60_seconds_ppg_rr_dalia.p','rb'))

In [ ]:
test_x = test_x.reshape(-1,30,1)

In [ ]:
test_embeddings = model.predict(test_x)
rank_1 = give_rank_accuracy(test_embeddings,test_y,rank = 1)
rank_5 = give_rank_accuracy(test_embeddings,test_y,rank = 5)
rank_10 = give_rank_accuracy(test_embeddings,test_y,rank = 10)
print(rank_1,rank_5,rank_10)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
print(classification_report(train_y,train_y_pred))
print(confusion_matrix(train_y,train_y_pred))

In [ ]:
tf.keras.layers.GaussianNoise(tf.random.uniform((30),minval=10,maxval=11))(test_x[0]),test_x[0]

In [ ]:
import multiprocessing

In [ ]:
from joblib import Parallel,delayed